In [1]:
! pip install transformers

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [6]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
import glob
import nltk

path_1 = 'russian_lit/ru_books_text_cleaner/copyrighted'
path_2 = 'russian_lit/ru_books_text_cleaner/non_copyrighted'
path_3 = 'russian_lit/ru_books_text_cleaner'

files_1 = glob.glob(path_1 + '/*.txt')
files_2 = glob.glob(path_2 + '/*.txt')
files_3 = glob.glob(path_3 + '/*.txt')
files = files_1 + files_2 + files_3

# create dict: translator to books
translator_to_sents = {}
for f in files:
    book_title = (f.split('/')[-1]).split('.')[0]
    book_name = book_title.split('_')[0]
    translator = book_title.split('_')[1]
    
    if translator not in translator_to_sents.keys():
        translator_to_sents[translator] = []

    with open(f, "r") as fp:
        book_text = fp.read()
        book_pars = book_text.split('\n')
        sents = []
        
        for par in book_pars:
            if len(par) > 40:
                par_sents = nltk.tokenize.sent_tokenize(par)
                long_sents = [s for s in par_sents if len(s) > 40]
                translator_to_sents[translator].extend(long_sents)

In [17]:
for k in translator_to_sents.keys():
    print(k, len(translator_to_sents[k]))

Garnett 70002
McDuff 39915
PV 68044
Katz 27795
Hogarth 8328


In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(list(translator_to_sents.keys()))
print(le.transform(["Garnett", "McDuff", "PV", "Katz", "Hogarth"]))

[0 3 4 2 1]


In [56]:
# for each translator
# split each book into sentences with sent_tokenizer
# drop sentences shorter than 6 words
# add to data dict: {'idx': 0, 'label': 1, 'sentence': "Our friends won't buy this analysis, let alone the next one we propose."}
data_list = []
i = 0
for tr in translator_to_sents.keys():
    label = le.transform([tr])[0]
    for s in translator_to_sents[tr]:
        sent_dict = {'idx': i, 'labels': label, 'sentence': s}
        data_list.append(sent_dict)
        i += 1

print(data_list[:10])

[{'idx': 0, 'labels': 0, 'sentence': "At nine o'clock in the morning, towards the end of November, the Warsaw train was approaching Petersburg at full speed."}, {'idx': 1, 'labels': 0, 'sentence': 'It was thawing, and so damp and foggy that it was difficult to distinguish anything ten paces from the line to right or left of the carriage windows.'}, {'idx': 2, 'labels': 0, 'sentence': 'Some of the passengers were returning from abroad, but the third-class compartments were most crowded, chiefly with people of humble rank, who had come a shorter distance on business.'}, {'idx': 3, 'labels': 0, 'sentence': "All of course were tired and shivering, their eyes were heavy after the night's journey, and all their faces were pale and yellow to match the fog."}, {'idx': 4, 'labels': 0, 'sentence': 'In one of the third-class carriages, two passengers had, from early dawn, been sitting facing one another by the window.'}, {'idx': 5, 'labels': 0, 'sentence': 'Both were young men, not very well dres

In [57]:
import pandas as pd 

df = pd.DataFrame(data_list)
df.head() 

,idx,labels,sentence
0,0,0,"At nine o'clock in the morning, towards the en..."
1,1,0,"It was thawing, and so damp and foggy that it ..."
2,2,0,Some of the passengers were returning from abr...
3,3,0,"All of course were tired and shivering, their ..."
4,4,0,"In one of the third-class carriages, two passe..."


In [92]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['sentence'].values.tolist(), df['labels'].values.tolist(),
    stratify = df['labels'], test_size= 0.3
)
print(train_labels[:5])
print('train size: ', len(list(train_labels)))
print('test size: ', len(list(test_labels)))

sentences = {}
sentences['train'] = []
sentences['test'] = []
for t, l in zip(train_texts, train_labels):
    datum = {'label': l, 'text': t}
    sentences['train'].append(datum)
for t, l in zip(test_texts, test_labels):
    datum = {'label': l, 'text': t}
    sentences['test'].append(datum)

print(sentences['train'][4])

[4, 1, 0, 0, 4]
train size:  149858
test size:  64226
{'label': 4, 'text': "I will note, anticipating events, that had it not been for Yulia Mikhailovna's selfimportance and ambition, perhaps none of the things these bad little people man aged to do here would have taken place."}


In [88]:
BERT_MODEL = "bert-base-multilingual-cased"

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

In [99]:
! pip install datasets

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [103]:
import datasets

train_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=sentences['train']))
test_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=sentences['test']))

In [105]:

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 64226/64226 [00:04<00:00, 14566.21 examples/s]


In [106]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [108]:
! pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [109]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [111]:
label_list = translator_to_sents.keys()
id_list = le.transform(list(label_list))

id2label = {}
label2id = {}
for l, i in zip(label_list, id_list):
    id2label[i] = l
    label2id[l] = i

print(id2label)
print(label2id)

{0: 'Garnett', 3: 'McDuff', 4: 'PV', 2: 'Katz', 1: 'Hogarth'}
{'Garnett': 0, 'McDuff': 3, 'PV': 4, 'Katz': 2, 'Hogarth': 1}


In [112]:
from numpy import mean
from torch import nn
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# device (turn on GPU acceleration for faster execution)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels = len(translator_to_sents.keys()))
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [114]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 149858
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 18734
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
trainer.evaluate()